Data from https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database/data

In [1]:
# !pip install oracledb

In [2]:
import oracledb
import pandas as pd

In [3]:
def connect_oracle():
    password = 'password'
    connection=oracledb.connect(
         config_dir="/opt/oracle/instantclient_21_12/network/admin",
         user="user",
         password=password,
         dsn="name_high",
         wallet_location="/opt/oracle/instantclient_21_12/network/admin",
         wallet_password=password)
    return connection

In [4]:
def Run_Query_Function(query):
    # Create a cursor object
    connection = connect_oracle()
    cursor = connection.cursor()
    
    # Execute a query
    cursor.execute(query)
    
    # Fetch the results
    results = cursor.fetchall()
    
    # Get the column names
    column_names = [i[0] for i in cursor.description]
    
    # Create a DataFrame from the results
    dataframe = pd.DataFrame(results, columns=column_names)
    
    # Close the cursor and connection
    cursor.close()
    connection.close()

    return dataframe


In [5]:
Run_Query_Function('''
    SELECT COUNT(DISTINCT A.NAME)
    FROM ANIME A
    ''')

,COUNT(DISTINCTA.NAME)
0,12292


In [6]:
Run_Query_Function('''
    SELECT COUNT(A.ANIME_ID)
    FROM ANIME A
    WHERE A.RATING IS NULL
    ''')

,COUNT(A.ANIME_ID)
0,230


In [7]:
Run_Query_Function('''
    SELECT cast(ROUND(A.RATING,0) as int) AS RATING_NUM,COUNT(A.ANIME_ID)
    FROM ANIME A
    WHERE A.RATING IS NOT NULL
    GROUP BY cast(ROUND(A.RATING,0) as int)
    ORDER BY cast(ROUND(A.RATING,0) as int)
    ''')

,RATING_NUM,COUNT(A.ANIME_ID)
0,2,6
1,3,79
2,4,445
3,5,1400
4,6,3712
5,7,4669
6,8,1616
7,9,134
8,10,3


In [8]:
Run_Query_Function('''
    SELECT A.TYPE,COUNT(A.ANIME_ID)
    FROM ANIME A
    GROUP BY A.TYPE
    ORDER BY A.TYPE
 ''')

,TYPE,COUNT(A.ANIME_ID)
0,Movie,2348
1,Music,488
2,ONA,659
3,OVA,3311
4,Special,1676
5,TV,3787
6,None,25


In [9]:
Run_Query_Function('''
    SELECT COUNT(A.ANIME_ID)
    FROM ANIME A
    WHERE LOWER(A.GENRE) LIKE 'hentai%'
 ''')

,COUNT(A.ANIME_ID)
0,947


In [27]:
Anime_Filtered = Run_Query_Function('''
SELECT A.ANIME_ID AS ANIME_ANIME_ID, 
        A.NAME, 
        A.GENRE, 
        A.TYPE, 
        A.EPISODES, 
        A.RATING AS ANIME_RATING, 
        A.MEMBERS
    FROM ANIME A
    WHERE A.RATING IS NOT NULL
    AND LOWER(A.GENRE) NOT LIKE 'hentai%'
    AND LOWER(A.NAME) NOT LIKE 'hentai%'
    ''')

In [28]:
Anime_Filtered.columns

Index(['ANIME_ANIME_ID', 'NAME', 'GENRE', 'TYPE', 'EPISODES', 'ANIME_RATING',
       'MEMBERS'],
      dtype='object')

In [29]:
n_anime = len(Anime_Filtered.ANIME_ANIME_ID.unique())
print("--------------------")
print("Number of Anime:",n_anime)

--------------------
Number of Anime: 11075


In [30]:
data = Anime_Filtered.copy()

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Assuming 'data' is your DataFrame obtained from the SQL query
# Create a TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Replace NaN with an empty string
data['GENRE'] = data['GENRE'].fillna('')

# Construct the TF-IDF matrix
tfidf_matrix = tfidf.fit_transform(data['GENRE'])

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Create a reverse mapping of indices and anime titles
indices = pd.Series(data.index, index=data['NAME']).to_dict()



In [46]:
# Function to get recommendations based on cosine similarity
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the anime that matches the title
    idx = indices[title]

    # Get the pairwise similarity scores of all anime with that anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the anime based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar anime
    sim_scores = sim_scores[1:11]

    # Get the anime indices
    anime_indices = [i[0] for i in sim_scores]
    
    anime_indices = [i for i in anime_indices if i != idx]
    
    # Return the top 10 most similar anime
    return data['NAME'].iloc[anime_indices]


In [49]:
get_recommendations('Naruto')

2465                                   Naruto: Shippuuden
3403    Boruto: Naruto the Movie - Naruto ga Hokage ni...
4943                                          Naruto x UT
5072          Naruto: Shippuuden Movie 4 - The Lost Tower
5173    Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...
6092    Naruto Soyokazeden Movie: Naruto to Mashin to ...
9009                 Naruto Shippuuden: Sunny Side Battle
8060                              Kyutai Panic Adventure!
2634           Naruto: Shippuuden Movie 6 - Road to Ninja
Name: NAME, dtype: object

In [50]:
Anime_Filtered.to_csv("Anime_Data_Final.csv",index=False)